In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import dotenv
import os
dotenv.load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",convert_system_message_to_human=False)

In [ ]:
import bs4
from langchain import hub

In [ ]:
from langchain.chains import create_retrieval_chain

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import MessagesPlaceholder

In [ ]:
loader = PyPDFLoader('D:\Hackathons\SAP-HAkathon\Fraud Suraksha\GMaps.pdf')

In [ ]:
doc = loader.load()

In [ ]:
doc

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)

In [ ]:
splits

In [ ]:
vectorstore = Chroma.from_documents(documents=splits, embedding=gemini_embeddings)
retriever = vectorstore.as_retriever()

In [ ]:
retriever

In [ ]:
system_prompt = (
    "You are an AI-powered fraud detection assistant designed to help users verify whether a person, message, or situation is fraudulent or safe. "
    "Use the following retrieved context to analyze scam patterns and assess the fraud risk score. "
    "If you don't have enough data to confirm, suggest caution and advise the user to be careful. "
    "Ask relevant follow-up questions if needed to gather more details before making a decision. "
    "Provide structured responses, including fraud risk score, detected red flags, and recommended actions. "
    "Never generate fake reports, financial guarantees, or unverifiable claims. "
    "\n\n"
    "{context}"
)  


In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answering_chain=create_stuff_documents_chain(model, chat_prompt)

In [ ]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [ ]:
response = rag_chain.invoke({"input":"What is the report contains"})

In [ ]:
response["answer"]

In [ ]:
response = rag_chain.invoke({"input":"Hii im dhanush"})

In [ ]:
response["answer"]

In [ ]:
response = rag_chain.invoke({"input":"What is my name"})
response["answer"]

### RAG With Memory

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage

In [ ]:
store = {}

In [ ]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
config = {
        "configurable": {"session_id": "abc123"}
    }

In [ ]:
conversational_rag_chain.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What is Google Maps Fraud?"},
    config=config,  # constructs a key "abc123" in `store`.
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": "My name is Dhanush"},
    config={
        "configurable": {"session_id": "session"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What is my name"},
    config={
        "configurable": {"session_id": "session"}
    },  
)["answer"]

In [ ]:
from langchain_core.messages import AIMessage

In [ ]:
for message in store["session"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

In [ ]:
from langchain_core.messages import AIMessage

In [ ]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

In [ ]:
store

In [ ]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

In [ ]:
conversational_rag_chain.invoke(
    {"input": "Hey i think i meat with a scammer...!"},
    config={"configurable": {"session_id": "session1"}},
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": "Encountered with the whats app and asked me to write reviews and give rating and asked me to send screenshot"},
    config={"configurable": {"session_id": "session1"}},
)["answer"]

In [ ]:
for message in store["session1"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

In [ ]:
conversational_rag_chain.invoke(
    {"input": "Hii im dhanush"},
    config={"configurable": {"session_id": "fraud"}},
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What is my name"},
    config={"configurable": {"session_id": "fraud"}},
)["answer"]